In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, pipeline, metrics
from sklearn.model_selection import GridSearchCV

In [2]:
# Define Metric (Quadratic Weighted Kappa)
# The following 3 functions are taken from https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [3]:
train_clean = pd.read_csv("./train_clean.csv")
train_clean = train_clean.fillna("")
train_clean.head()

,query,product_title,product_description,median_relevance,relevance_variance
0,bridal shower decoration,accent pillow heart design red black,red satin accent pillow embroider heart black ...,1,0.000
1,lead christmas light,set battery operate multi lead train christmas...,set battery operate train christmas light item...,4,0.000
2,projector,viewsonic pro dlp multimedia projector,,4,0.471
3,wine rack,concept housewares wr solid wood ceiling wall ...,like silent sturdy tree southern enterprise bi...,4,0.000
4,light bulb,wintergreen light christmas lead light bulb pack,wtgr feature nickel base average hour acrylic ...,2,0.471


In [4]:
tmp = train_clean["query"] + [" "] + train_clean["product_title"]
traindata = tmp.values.tolist()
y = train_clean["median_relevance"].values

test_clean = pd.read_csv("./test_clean.csv")
test_clean = test_clean.fillna("")
tmp = test_clean["query"] + [" "] + test_clean["product_title"]
testdata = tmp.values.tolist()

### Count Vectorizer

In [5]:
ctv = CountVectorizer(analyzer = "word", \
                      tokenizer = None, \
                      preprocessor = None, \
                      stop_words = None, \
                      max_features = None) 

ctv.fit(traindata)
X =  ctv.transform(traindata) 
X_test = ctv.transform(testdata)

In [6]:
# Initialize SVD
svd = TruncatedSVD()
# Random Forest
rf_model = RandomForestClassifier()

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('rf', rf_model)])

# Create a parameter grid to search for best parameters for everything in the pipeline
param_grid = {'svd__n_components' : [200, 400],
              'rf__n_estimators':[10, 30]}

In [7]:
# Kappa Scorer 
kappa_scorer = metrics.make_scorer(quadratic_weighted_kappa, greater_is_better = True)

# Initialize Grid Search Model
model = GridSearchCV(estimator = clf, param_grid=param_grid, scoring=kappa_scorer,
                     verbose=10, n_jobs=-1, cv=2)

# Fit Grid Search Model
model.fit(X, y)
print("Best score: %0.4f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Get best model
best_model = model.best_estimator_

# Fit model with best parameters optimized for quadratic_weighted_kappa
best_model.fit(X,y)
preds = best_model.predict(X_test)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] rf__n_estimators=10, svd__n_components=200 ......................
[CV] rf__n_estimators=10, svd__n_components=200 ......................
[CV] rf__n_estimators=10, svd__n_components=400 ......................
[CV] rf__n_estimators=10, svd__n_components=400 ......................
[CV] rf__n_estimators=30, svd__n_components=200 ......................
[CV] rf__n_estimators=30, svd__n_components=200 ......................
[CV] rf__n_estimators=30, svd__n_components=400 ......................
[CV] rf__n_estimators=30, svd__n_components=400 ......................
[CV]  rf__n_estimators=10, svd__n_components=200, score=0.3422331836436202, total=  10.1s
[CV]  rf__n_estimators=10, svd__n_components=200, score=0.35695620799726624, total=  10.1s


[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:   10.3s remaining:   30.8s


[CV]  rf__n_estimators=30, svd__n_components=200, score=0.33750679196636524, total=  12.7s


[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:   12.9s remaining:   21.5s


[CV]  rf__n_estimators=30, svd__n_components=200, score=0.3529915884752872, total=  13.3s


...

[CV]  rf__n_estimators=10, svd__n_components=400, score=0.28896136122998306, total=  14.3s
[CV]  rf__n_estimators=10, svd__n_components=400, score=0.28092957746478875, total=  14.4s


[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:   14.4s remaining:    8.6s
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:   14.5s remaining:    4.8s


[CV]  rf__n_estimators=30, svd__n_components=400, score=0.2680577535005765, total=  16.3s
[CV]  rf__n_estimators=30, svd__n_components=400, score=0.2652465027871209, total=  16.4s


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   16.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   16.6s finished


Best score: 0.3496
Best parameters set:
	rf__n_estimators: 10
	svd__n_components: 200


In [9]:
# Create submission file
test = pd.read_csv("./data/test.csv")
idx = test["id"].values
submission = pd.DataFrame({"id": idx, "prediction": preds})
submission.to_csv("CTV_RF.csv", index=False)

Private leader board score 0.43416.

## TFIDF

In [10]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
        strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
        ngram_range=(1, 5), use_idf=1,smooth_idf=1,sublinear_tf=1,
        stop_words = 'english')

tfv.fit(traindata)
X =  tfv.transform(traindata) 
X_test = tfv.transform(testdata)

In [11]:
# Initialize SVD
svd = TruncatedSVD()
# Random Forest
rf_model = RandomForestClassifier()

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('rf', rf_model)])

# Create a parameter grid to search for best parameters for everything in the pipeline
param_grid = {'svd__n_components' : [200, 400],
              'rf__n_estimators':[10, 30]}

In [12]:
# Kappa Scorer 
kappa_scorer = metrics.make_scorer(quadratic_weighted_kappa, greater_is_better = True)

# Initialize Grid Search Model
model = GridSearchCV(estimator = clf, param_grid=param_grid, scoring=kappa_scorer,
                     verbose=10, n_jobs=-1, cv=2)

# Fit Grid Search Model
model.fit(X, y)
print("Best score: %0.4f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Get best model
best_model = model.best_estimator_

# Fit model with best parameters optimized for quadratic_weighted_kappa
best_model.fit(X,y)
preds = best_model.predict(X_test)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] rf__n_estimators=10, svd__n_components=200 ......................
[CV] rf__n_estimators=10, svd__n_components=200 ......................
[CV] rf__n_estimators=10, svd__n_components=400 ......................
[CV] rf__n_estimators=10, svd__n_components=400 ......................
[CV] rf__n_estimators=30, svd__n_components=200 ......................
[CV] rf__n_estimators=30, svd__n_components=200 ......................
[CV] rf__n_estimators=30, svd__n_components=400 ......................
[CV] rf__n_estimators=30, svd__n_components=400 ......................
[CV]  rf__n_estimators=10, svd__n_components=200, score=0.394493188112357, total=  12.9s
[CV]  rf__n_estimators=10, svd__n_components=200, score=0.3965538285154473, total=  13.1s


[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:   13.2s remaining:   39.7s


[CV]  rf__n_estimators=30, svd__n_components=200, score=0.37280817378435327, total=  15.8s


[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:   16.0s remaining:   26.7s


[CV]  rf__n_estimators=30, svd__n_components=200, score=0.3743110464201195, total=  16.0s


...

[CV]  rf__n_estimators=10, svd__n_components=400, score=0.3447000113822366, total=  18.4s
[CV]  rf__n_estimators=10, svd__n_components=400, score=0.36326079267528255, total=  18.4s


[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:   18.6s remaining:   11.1s
[Parallel(n_jobs=-1)]: Done   6 out of   8 | elapsed:   18.6s remaining:    6.2s


[CV]  rf__n_estimators=30, svd__n_components=400, score=0.3192030323729018, total=  20.6s
[CV]  rf__n_estimators=30, svd__n_components=400, score=0.34078530726515943, total=  20.9s


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   21.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:   21.2s finished


Best score: 0.3955
Best parameters set:
	rf__n_estimators: 10
	svd__n_components: 200


In [13]:
# Create submission file
submission = pd.DataFrame({"id": idx, "prediction": preds})
submission.to_csv("TFIDF_RF.csv", index=False)

Private leader board score 0.45534.